The goal of this function is to identify subgroups of customers based on behavior patterns.  We identified groups based on magnitude of average daily use and on response to weather (as defined by k-shape clustering over sept-feb period)

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import time

In [2]:
# define data location
location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
#location = '/Users/loki/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'

In [3]:
readdata = pd.read_pickle(location+'peco_daily.pkl.zip')
# readdata = readdata[['DMETERNO','TARIFF','REVENUCODE']].drop_duplicates()
readdata.head()

,ID,Dt,Use,DACCOUNTID,DCUSTOMERID,DMETERNO,TARIFF,FUELTYPE,REVENUCODE,Weekday,Holiday
495153,"(68881639149832, 606395738288)",2017-10-01,1.00,68881639149832,3188526427696,606395738288,GH0,GAS,1.0,0,0
482563,"(68690879683500, 608923719096)",2017-10-01,0.00,68690879683500,19731696176432,608923719096,GH0,GAS,1.0,0,0
1063222,"(164788222898932, 640944705832)",2017-10-01,1.14,164788222898932,33149958808264,640944705832,GH0,GAS,1.0,0,0
482201,"(68689501197696, 726789359116)",2017-10-01,2.00,68689501197696,756418245856,726789359116,GH0,GAS,1.0,0,0
160918,"(20628346369984, 640143121596)",2017-10-01,4.12,20628346369984,24713480259220,640143121596,UHC,GAS,3.0,0,0


In [4]:
# import kshape cluster information from other script
clusters = pd.read_excel(location+'all.xlsx')
clusters = clusters[['DMETERNO','Size','Lable']]
clusters.head()

,DMETERNO,Size,Lable
0,2575037524724,Huge,0
1,2574628679116,Huge,0
2,3881639196592,Huge,0
3,2769160671848,Huge,0
4,2613828993176,Huge,0


In [5]:
clusters = clusters[['DMETERNO','Size','Lable']]

clusters['Cluster'] = None
clusters.loc[ (clusters.Size == 'Huge') & (clusters.Lable == 0), 'Cluster'] = 'Huge_NoResp' 
clusters.loc[ (clusters.Size == 'Huge') & (clusters.Lable == 1), 'Cluster'] = 'Huge_Resp' 

clusters.loc[ (clusters.Size == 'High') & (clusters.Lable == 0), 'Cluster'] = 'High_Resp' 
clusters.loc[ (clusters.Size == 'High') & (clusters.Lable == 1), 'Cluster'] = 'High_Resp' 
clusters.loc[ (clusters.Size == 'High') & (clusters.Lable == 2), 'Cluster'] = 'High_NoResp' 

clusters.loc[ (clusters.Size == 'Moderate') & (clusters.Lable == 0), 'Cluster'] = 'Mod_Resp' 
clusters.loc[ (clusters.Size == 'Moderate') & (clusters.Lable == 1), 'Cluster'] = 'Mod_Resp' 
clusters.loc[ (clusters.Size == 'Moderate') & (clusters.Lable == 2), 'Cluster'] = 'Mod_NoResp' 

clusters.loc[ (clusters.Size == 'Light'), 'Cluster'] = 'Light' 

clusters.loc[ (clusters.Size == 'Minimal') & (clusters.Lable == 0), 'Cluster'] = 'Min_Resp' 
clusters.loc[ (clusters.Size == 'Minimal') & (clusters.Lable == 1), 'Cluster'] = 'Min_Resp' 
clusters.loc[ (clusters.Size == 'Minimal') & (clusters.Lable == 2), 'Cluster'] = 'Min_NoResp' 

clusters.head()

,DMETERNO,Size,Lable,Cluster
0,2575037524724,Huge,0,Huge_NoResp
1,2574628679116,Huge,0,Huge_NoResp
2,3881639196592,Huge,0,Huge_NoResp
3,2769160671848,Huge,0,Huge_NoResp
4,2613828993176,Huge,0,Huge_NoResp


In [6]:
codebook = readdata[['DMETERNO','TARIFF','REVENUCODE']].drop_duplicates().merge(clusters, how='inner',on='DMETERNO')
codebook.head()

,DMETERNO,TARIFF,REVENUCODE,Size,Lable,Cluster
0,606395738288,GH0,1.0,Minimal,1,Min_Resp
1,608923719096,GH0,1.0,Minimal,1,Min_Resp
2,640944705832,GH0,1.0,Light,1,Light
3,726789359116,GH0,1.0,Moderate,1,Mod_Resp
4,640143121596,UHC,3.0,Moderate,2,Mod_NoResp


In [8]:
codebook.to_pickle(location+'final_codebook_combined_v2.pkl.zip')
codebook.to_csv(location+'final_codebook_combined_v2.csv')

,ID,Dt,Use,DACCOUNTID,DCUSTOMERID,DMETERNO,TARIFF,FUELTYPE,REVENUCODE,Weekday,Holiday
495153,"(68881639149832, 606395738288)",2017-10-01,1.00,68881639149832,3188526427696,606395738288,GH0,GAS,1.0,0,0
482563,"(68690879683500, 608923719096)",2017-10-01,0.00,68690879683500,19731696176432,608923719096,GH0,GAS,1.0,0,0
1063222,"(164788222898932, 640944705832)",2017-10-01,1.14,164788222898932,33149958808264,640944705832,GH0,GAS,1.0,0,0
482201,"(68689501197696, 726789359116)",2017-10-01,2.00,68689501197696,756418245856,726789359116,GH0,GAS,1.0,0,0
160918,"(20628346369984, 640143121596)",2017-10-01,4.12,20628346369984,24713480259220,640143121596,UHC,GAS,3.0,0,0


In [14]:
# make centroids sheet
weather = pd.read_pickle(location+'daily_weather.pkl.zip')

readdata['DMETERNO'] = readdata['DMETERNO'].astype(str)
codebook['DMETERNO'] = codebook['DMETERNO'].astype(str)

data = readdata[['DMETERNO','Dt','Use']].pivot_table(values='Use', index='DMETERNO', columns='Dt').reset_index()
data = pd.merge(data, codebook[['DMETERNO','Cluster']], how='inner', on='DMETERNO')
data = data.groupby('Cluster').mean().reset_index()

centroids = pd.melt(data, id_vars='Cluster',var_name='Dt', value_name='Use')
centroids['Dt'] =  pd.to_datetime(centroids['Dt'])
centroids = centroids.dropna()

centroids_clust = pd.merge(centroids, weather, how='inner', on='Dt')

centroids_clust.to_csv('/Users/loki/Desktop/centroids.csv')
del(data, centroids, centroids_clust)

,DMETERNO,TARIFF,REVENUCODE,Size,Lable,Cluster
0,606395738288,GH0,1.0,Minimal,1,Min_Resp
1,608923719096,GH0,1.0,Minimal,1,Min_Resp
2,640944705832,GH0,1.0,Light,1,Light
3,726789359116,GH0,1.0,Moderate,1,Mod_Resp
4,640143121596,UHC,3.0,Moderate,2,Mod_NoResp


In [7]:
# how many of each cluster?
codebook.groupby('Cluster').size().reset_index()

,Cluster,0
0,High_NoResp,11
1,High_Resp,348
2,Huge_NoResp,9
3,Huge_Resp,14
4,Light,3551
5,Min_NoResp,1084
6,Min_Resp,1274
7,Mod_NoResp,12
8,Mod_Resp,654


In [8]:
# what is percent daily use provided by each cluster?
data = pd.read_pickle(location+'peco_daily.pkl.zip')
data = data.merge(clusters, how='inner',on='DMETERNO')
data = data[['DMETERNO','Dt','Use','Cluster']]
data = data[data['Dt']>='2018-01-01']

In [9]:
data = data.groupby('Cluster').sum().reset_index()
data['PctUse']=data.Use/sum(data.Use)
data

,Cluster,DMETERNO,Use,PctUse
0,High_NoResp,7569556168042952,103907.70,0.009531
1,High_Resp,205768960536410080,2716794.48,0.249192
2,Huge_NoResp,5677856220885840,3555672.66,0.326137
3,Huge_Resp,10089003415485828,1206625.90,0.110675
4,Light,1065377576259800064,2136071.27,0.195927
5,Min_NoResp,465830286221511488,74836.40,0.006864
6,Min_Resp,625300284458578048,263542.60,0.024173
7,Mod_NoResp,5895836854693432,25108.89,0.002303
8,Mod_Resp,242438556670432000,819832.88,0.075198


In [10]:
(codebook[['Size','Lable','TARIFF']].groupby(['Size','Lable']).count()
 .pivot_table(index='Lable',columns=['Size'],values='TARIFF'))
# (codebook[['UseGroup','ColdGroup','TARIFF']].groupby(['UseGroup','ColdGroup']).count()
#  .pivot_table(index='ColdGroup',columns=['UseGroup'],values='TARIFF')).to_csv(location+'groups_clusters.csv')
# print(codebook[['Cluster','TARIFF']].groupby('Cluster').count())

Size,High,Huge,Light,Minimal,Moderate
Lable,,,,,
0,59.0,9.0,NaN,288.0,56.0
1,289.0,14.0,3551.0,986.0,598.0
2,11.0,NaN,NaN,1084.0,12.0


In [11]:
# crosstab tariff vs cluster
pd.crosstab(codebook.TARIFF,codebook.Cluster)
#pd.crosstab(codebook.TARIFF,codebook.Cluster).to_csv(location+'pivot_clusters.csv')

Cluster,High_NoResp,High_Resp,Huge_NoResp,Huge_Resp,Light,Min_NoResp,Min_Resp,Mod_NoResp,Mod_Resp
TARIFF,,,,,,,,,
GC0,0,50,0,3,8,10,13,1,25
GCE,0,2,0,0,0,0,0,0,0
GH0,0,1,0,0,2634,998,980,0,340
GH5,0,0,0,0,4,0,0,0,0
GH8,0,0,0,0,33,6,10,0,2
GHC,8,151,0,1,104,25,56,9,126
GHE,0,6,0,0,0,0,0,0,2
GR0,0,0,0,0,103,13,66,0,14
GR5,0,0,0,0,0,0,1,0,1


In [12]:
# crosstab tariff vs cluster
pd.crosstab(codebook.REVENUCODE,codebook.Cluster)
#pd.crosstab(codebook.REVENUCODE,codebook.Cluster).to_csv(location+'pivot_clusters.csv')

Cluster,High_NoResp,High_Resp,Huge_NoResp,Huge_Resp,Light,Min_NoResp,Min_Resp,Mod_NoResp,Mod_Resp
REVENUCODE,,,,,,,,,
1.0,0,1,0,0,3354,747,1167,0,425
3.0,10,339,0,10,126,36,80,12,213
5.0,1,7,9,4,0,0,1,0,0
12.0,0,0,0,0,50,299,20,0,13


## Electric rate code ##

In [3]:
# load customer data
codebook =  pd.read_pickle(location+'final_codebook_combined_v2.pkl.zip')
customer_in = pd.read_excel(location+'PECO Zip Customer 2018.10.01 v2.xlsx', sheet_name="Account")
customer_in = customer_in.drop(columns=['CITY', 'STATE', 'ZIPCODE', 'COUNTYCODE'])

In [5]:
electric = customer_in.loc[customer_in['FUELTYPE']=='ELECTRIC']
electric['electric_TARIFF'] = electric['TARIFF']
electric.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,DACCOUNTID,DCUSTOMERID,DMETERNO,TARIFF,FUELTYPE,REVENUCODE,electric_TARIFF
0,156414999216,20325174427256,5472978430796,ER0,ELECTRIC,12.0,ER0
3,432330788012,20325174427256,5472996672436,ER0,ELECTRIC,12.0,ER0
6,504636767772,30156886507608,22374171608192,ER0,ELECTRIC,1.0,ER0
7,585291286356,20325174427256,5473038783456,ER0,ELECTRIC,12.0,ER0
10,585291635664,28819424483052,5473038783456,ER0,ELECTRIC,1.0,ER0


In [6]:
# add accountID to codebook
codebook['DMETERNO'] = codebook['DMETERNO'].astype(str)
customer_in['DMETERNO'] = customer_in['DMETERNO'].astype(str)
codebook = codebook.merge(customer_in[['DACCOUNTID','DCUSTOMERID','DMETERNO']], how='inner', on='DMETERNO')
codebook.head()

,DMETERNO,TARIFF,REVENUCODE,Size,Lable,Cluster,DACCOUNTID,DCUSTOMERID
0,606395738288,GH0,1.0,Minimal,1,Min_Resp,68881639149832,3188526427696
1,608923719096,GH0,1.0,Minimal,1,Min_Resp,68690879683500,19731696176432
2,640944705832,GH0,1.0,Light,1,Light,164788222898932,33149958808264
3,726789359116,GH0,1.0,Moderate,1,Mod_Resp,68689501197696,756418245856
4,640143121596,UHC,3.0,Moderate,2,Mod_NoResp,20628346369984,24713480259220


In [7]:
# merge on accountID b/c DMETERNO is different for gas and electric meters
codebook['DACCOUNTID'] = codebook['DACCOUNTID'].astype(str)
electric['DACCOUNTID'] = electric['DACCOUNTID'].astype(str)

tmp = codebook.merge(electric[['DACCOUNTID','DMETERNO','electric_TARIFF']], how='inner', on='DACCOUNTID').drop_duplicates()
tmp.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,DMETERNO_x,TARIFF,REVENUCODE,Size,Lable,Cluster,DACCOUNTID,DCUSTOMERID,DMETERNO_y,electric_TARIFF
0,606395738288,GH0,1.0,Minimal,1,Min_Resp,68881639149832,3188526427696,4830210764136,ER0
1,608923719096,GH0,1.0,Minimal,1,Min_Resp,68690879683500,19731696176432,4884727147260,ER0
2,640944705832,GH0,1.0,Light,1,Light,164788222898932,33149958808264,4737889319540,ER0
3,726789359116,GH0,1.0,Moderate,1,Mod_Resp,68689501197696,756418245856,4830054856332,UR0
4,640143121596,UHC,3.0,Moderate,2,Mod_NoResp,20628346369984,24713480259220,4829934189824,UM2


In [8]:
tmp = tmp.drop_duplicates()
pd.crosstab(tmp.electric_TARIFF,tmp.Cluster)

Cluster,High_NoResp,High_Resp,Huge_Resp,Light,Min_NoResp,Min_Resp,Mod_NoResp,Mod_Resp
electric_TARIFF,,,,,,,,
CE8,0,0,0,2,0,0,0,0
CR8,0,0,0,40,8,16,0,4
EA2,0,1,0,0,0,0,0,0
EC0,0,0,0,4,2,3,0,1
EM2,1,55,1,61,17,32,1,66
EM7,1,5,0,0,0,1,0,2
ER0,0,1,0,2117,716,616,0,260
ERH,0,0,0,15,7,13,0,2
UA2,0,6,0,0,1,0,0,1


In [8]:
# who are huge users?
codebook.loc[codebook['Size']=='Huge'][['DCUSTOMERID','DACCOUNTID','DMETERNO','Cluster']].to_csv('/Users/loki/Desktop/gas_huge_users.csv')

In [9]:
# who are minimal-use non-responders?
codebook.loc[codebook['Cluster']=='Min_NoResp'][['DCUSTOMERID','DACCOUNTID','DMETERNO','Cluster']].to_csv('/Users/loki/Desktop/gas_min_nonresp.csv')

In [45]:
# who are minimal-use non-responders without electric heat?
tmp.loc[(tmp['Cluster']=='Min_NoResp') & 
        ((tmp['electric_TARIFF']=='ER0') | (tmp['electric_TARIFF']=='UR0') | 
         (tmp['electric_TARIFF']=='CR8'))].to_csv('/Users/loki/Desktop/gas_min_nonresp_elec-nonheat.csv')

In [9]:
# what are gas rate codes for minimal-use non-responders without electric heat?
subset=tmp.loc[(tmp['Cluster']=='Min_NoResp')]
pd.crosstab(subset.electric_TARIFF,subset.TARIFF)

TARIFF,GC0,GH0,GH8,GHC,GR0,GR8,UGH,UGR,UHC
electric_TARIFF,,,,,,,,,
CR8,0,0,6,0,0,2,0,0,0
EC0,0,0,0,2,0,0,0,0,0
EM2,2,0,0,15,0,0,0,0,0
ER0,0,703,0,0,11,0,2,0,0
ERH,0,7,0,0,0,0,0,0,0
UA2,0,0,0,0,0,0,0,0,1
UM2,0,0,0,6,0,0,0,0,0
UR0,0,47,0,0,4,0,22,3,0
